In [53]:
import numpy as np
import os

import plugins.custom_config as custom_config
import plugins.bot_core as bot_core
import plugins.tool_dice as tool_dice
import plugins.info_help as info_help
import plugins.info_game as info_game
import plugins.utils as utils

from plugins import bot

b = bot

In [54]:
assert utils.ChineseToEnglishSymbol('。') == '.'
assert utils.ChineseToEnglishSymbol('.') == '.'
assert utils.int2str(4) == '+4'
assert utils.int2str(0) == '+0'
assert utils.int2str(-1) == '-1'

assert await bot_core.ParseInput('.rd20')
assert await bot_core.ParseInput(' . r D20 reason')
assert await bot_core.ParseInput('.r')
assert await bot_core.ParseInput('.rh')
assert await bot_core.ParseInput('.rs')
assert await bot_core.ParseInput('abc') is None
assert await bot_core.ParseInput('') is None
assert await bot_core.ParseInput('测试') is None

In [55]:
assert tool_dice.RollDiceCommand('12') [1] == '12=12'
assert tool_dice.RollDiceCommand('4d6+2')[0] == 0 and tool_dice.RollDiceCommand('4d6+2') [1][:6] == '4D6+2='
assert tool_dice.RollDiceCommand('d20') [0] == 0 and tool_dice.RollDiceCommand('d20') [1][:3] == 'D20'
assert tool_dice.RollDiceCommand('d') [0] == 0 and tool_dice.RollDiceCommand('d') [1][:3] == 'D20'
assert tool_dice.RollDiceCommand('1D4') [0] == 0
assert tool_dice.RollDiceCommand('30D4') [0] == 0
assert tool_dice.RollDiceCommand('D20优势+2') [0] == 0 and tool_dice.RollDiceCommand('D20优势+2') [1][:12] == 'D20优势+2=Max('
assert tool_dice.RollDiceCommand('D20劣势-4') [0] == 0 and tool_dice.RollDiceCommand('D20劣势-4') [1][:12] == 'D20劣势-4=Min('
assert tool_dice.RollDiceCommand('4d6k3') [0] == 0 and tool_dice.RollDiceCommand('4d6k3') [1][:11] == '4D6K3=Max3('
assert tool_dice.RollDiceCommand('6#4d6k3') [0] == 0 and tool_dice.RollDiceCommand('6#4d6k3') [1][:14] == '6次4D6K3={\nMax3'
assert tool_dice.RollDiceCommand('2d20k1+5+1d4') [0] == 0 and tool_dice.RollDiceCommand('2d20k1+5+1d4') [1][:17] == '2D20K1+5+1D4=Max('
assert tool_dice.RollDiceCommand('d优势+2')[1][:12] == 'D20优势+2=Max('
assert tool_dice.RollDiceCommand('3dk1+5抗性')[1][:16] == '3D20K1+5抗性=(Max('

In [56]:
assert tool_dice.SplitDiceCommand('d20优势+5攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5     攻击地精') == ('d20优势+5', '攻击地精')
assert tool_dice.SplitDiceCommand('d20优势+5') == ('d20优势+5', '')

In [57]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd     ', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd    未知原因', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd20攻击', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd优势', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rd+5', '821480843', 'TEST_USER'))[0].resultStr
assert (await b.ProcessMessage('.rs6d20', '82148084', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.rs4#6d20', '82148084', 'TEST_USER', 'groupId_test'))[0].resultStr

In [58]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.nn 西瓜', '821480843', 'ABCD'))[0].resultStr == '要称呼你为西瓜吗? 没问题!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:9] == '西瓜掷出了D20='
assert (await b.ProcessMessage('.nn 冬瓜', '821480843', 'ABCD'))[0].resultStr == '要称呼你为冬瓜吗? 没问题!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:9] == '冬瓜掷出了D20='
assert ((await b.ProcessMessage('.jrrp', '821480843', 'TEST_USER'))[0].resultStr)
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.rd20', '821480843', 'TEST_USER'))[0].resultStr[:16] == 'TEST_USER掷出了D20='
assert ((await b.ProcessMessage('.jrrp', '821480843', 'TEST_USER'))[0].resultStr)

In [59]:
assert (await b.ProcessMessage('.nn', '821480843', 'ABCD'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.nn 格鲁特', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '要称呼你为格鲁特吗? 没问题!'
assert (await b.ProcessMessage('.hp 40', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '成功将格鲁特的生命值设置为40=40'
assert (await b.ProcessMessage('.hp 40/50', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '成功将格鲁特的生命值设置为40=40, 最大生命值是50'
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '还没有做好先攻列表哦'
assert (await b.ProcessMessage('.ri+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:12] == '格鲁特先攻:D20+2='
assert (await b.ProcessMessage('.ri-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:12] == '格鲁特先攻:D20-1='
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:10] == '格鲁特先攻:D20='
assert (await b.ProcessMessage('.ri-2地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '地精先攻:D20-2='
assert (await b.ProcessMessage('.ri+1 1d4+1#大僵尸/小僵尸/快速僵尸+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri劣势+1d4 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:19] =='地精先攻:D20劣势+1D4=Min('
assert (await b.ProcessMessage('.ri20 巢穴', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '巢穴先攻:20=20'
assert (await b.ProcessMessage('.ri-5', '87654321', '米达尔', 'groupId_test'))[0].resultStr[:12] == '米达尔先攻:D20-5='
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init del 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经将地精从先攻列表中删除'
assert (await b.ProcessMessage('.init del米达尔', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经将米达尔从先攻列表中删除'
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '还没有做好先攻列表哦'
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'
assert (await b.ProcessMessage('.rd20', '821480843', 'ABCD'))[0].resultStr[:11] == 'ABCD掷出了D20='
assert (await b.ProcessMessage('.hp clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == '已经忘记了TEST_USER的生命值...'

In [60]:
assert (await b.ProcessMessage('.nn 格鲁特', '821480843', 'TEST_USER'))[0].resultStr == '要称呼你为格鲁特吗? 没问题!'
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri', '87654321', '米达尔', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 地精', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 卓尔精灵A', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 20/40', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 地精- 1d8+4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 格+ 1d8+3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp A-1d6+10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hpA2d8', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hpA-20', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp米-3d4+3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.米-3d4+3hp', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.地精hp- 1d8+4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.米/地精hp-4d8', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp clr', '87654321', '米达尔', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER'))[0].resultStr == '要用本来的名字称呼你吗? 了解!'

In [61]:
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.hp 20', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.ri 约翰', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn 约瑟夫', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn 约翰', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '12345678', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.init clr', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [62]:
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test')) == None
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.bot off', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test')) == None
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test', True))[0] != None
assert (await b.ProcessMessage('.bot on', '821480843', 'TEST_USER', 'groupId_test', True))[0].resultStr
assert (await b.ProcessMessage('.rd20沉默', '821480843', 'TEST_USER', 'groupId_test'))[0] != None

In [63]:
assert (await b.ProcessMessage('.dnd', '821480843', 'TEST_USER'))[0]
assert (await b.ProcessMessage('.dnd 3', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.dnd3 大力出奇迹', '821480843', 'TEST_USER', 'groupId_test'))[0]

In [64]:
assert (await b.ProcessMessage('.help', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_STR
assert (await b.ProcessMessage('.help指令', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_COMMAND_STR
assert (await b.ProcessMessage('.help  指 令', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_COMMAND_STR
assert (await b.ProcessMessage('.help链接', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_LINK_STR
assert (await b.ProcessMessage('.help协议', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_AGREEMENT_STR
assert (await b.ProcessMessage('.help查询', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_COMMAND_QUERY_STR
assert (await b.ProcessMessage('.help draw', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr == info_help.HELP_COMMAND_DRAW_STR

In [65]:
assert (await b.ProcessMessage('.查询火球术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询  火球术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询  火 球 术', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'
assert (await b.ProcessMessage('.查询火球', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr.find('找到多个匹配的条目:') != -1
assert (await b.ProcessMessage('.查询   ', '821480843', 'TEST_USER', 'groupId_test'))[0]
assert (await b.ProcessMessage('.查询 高/隐', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:13]
assert (await b.ProcessMessage('.查询 火球', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:11] == '火球术Fireball'

In [66]:
assert (await b.ProcessMessage('.rhd20优势', '821480843', 'TEST_USER', 'groupId_test'))[0].personIdList == ['821480843']
assert (await b.ProcessMessage('.rhd20优势', '821480843', 'TEST_USER', 'groupId_test'))[0].groupIdList is None
assert (await b.ProcessMessage('.r1d8抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r1d8+2d10抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r20易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.rd+2d8+1d4易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r3#1d8+1d12抗性', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r易伤d+2d8+1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.r3dk1易伤', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [67]:
assert (await b.ProcessMessage('.draw 狂野', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.draw狂野魔法浪涌表', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.draw', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.draw万象', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw天气', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw库藏宝藏cr0', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.draw11#库藏宝藏cr0', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)

In [68]:
assert (await b.ProcessMessage('.烹饪', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪-4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪优势+1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 经典/主菜', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 野炊', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.烹饪 经典/主菜////精灵/黑暗/奢侈', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [69]:
assert (await b.ProcessMessage('.点菜', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜1d4', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜3', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜5经典', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.点菜 经典/主菜////精灵/黑暗/奢侈', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.今日菜单', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [70]:
template = (await b.ProcessMessage('.角色卡模板', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage(f'.记录角色卡 {template}', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.法术位', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.2环法术位-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.1环-1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.金钱-52sp-20cp', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.金钱-5', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage(f'.金钱', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.查看角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.清除法术位', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.清除金钱', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.查看角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.完整角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.敏捷检定+4+1d4 躲藏', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.3#敏捷检定+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.3#敏捷攻击', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.3#敏捷豁免', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.敏捷豁免+2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.察觉检定', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert ((await b.ProcessMessage('.敏捷攻击', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.清除角色卡 ', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.nn', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.角色卡', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.说服检定', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.长休', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [71]:
assert (await b.ProcessMessage('.credit 12345678:+10', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.credit 12345678:-10', '821480843', 'TEST_USER', 'groupId_test'))[1].resultStr
assert ((await b.ProcessMessage('.好感度', '12345678', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.好感度', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.credit 821480843', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert ((await b.ProcessMessage('.dp', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert (await b.ProcessMessage('.今日笑话', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.今日笑话', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.明日笑话', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.昨日笑话', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [72]:
assert (await b.ProcessMessage('.记录笔记 包子', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 馒头', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 1:面条', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 1:+鸡蛋', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.记录笔记 1:+豆浆油条', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记 xxyyzz', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记 临时', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.清除笔记1', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.清除笔记 所有笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.笔记', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [73]:
assert (await b.ProcessMessage('伊丽莎白是谁', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('the shadow!!!!', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('请伊丽莎白保佑我啊!', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [74]:
assert (await b.ProcessMessage('.索引 守序邪恶', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.索引 施法时间', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.索引 长弓', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('2', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr[:2] == '长弓'

In [75]:
assert (await b.ProcessMessage('.name', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.name10#男性精灵', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('.name 10#女性人类', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [76]:
assert (await b.ProcessMessage('.welcome 欢迎~', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert b.GetWelcome('groupId_test')

In [77]:
assert (await b.ProcessMessage('.答题 科目一', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('f', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('T', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage('q', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr

In [87]:
template = (await b.ProcessMessage('.角色卡模板', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage(f'.记录角色卡 {template}', '821480843', 'TEST_USER1', 'groupId_test'))[0].resultStr
assert (await b.ProcessMessage(f'.记录角色卡 {template}', '00000000', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert(await b.ProcessMessage('.加入队伍 DND5E', '821480843', 'TEST_USER1', 'groupId_test'))[0].resultStr
assert(await b.ProcessMessage('.加入队伍 DND5E', '00000000', 'TEST_USER2', 'groupId_test'))[0].resultStr
assert((await b.ProcessMessage('.队伍点名', '821480843', 'TEST_USER', 'groupId_test'))[0].resultStr)
assert(await b.ProcessMessage('.清除角色卡', '00000000', 'TEST_USER1', 'groupId_test'))[0].resultStr
assert(await b.ProcessMessage('.清除角色卡', '821480843', 'TEST_USER2', 'groupId_test'))[0].resultStr